In [1]:
%load_ext autoreload
%autoreload 2

import numpy as npa
from matplotlib import pyplot
import os
from copy import deepcopy

from time import time

from math import ceil
from scipy.stats import spearmanr, gamma, poisson
import scipy.sparse as sp

from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from torch import tensor
from torch.cuda import is_available

from sciPENN.sciPENN_API import sciPENN_API

ModuleNotFoundError: No module named 'sciPENN'

In [ ]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

In [ ]:
"""Read in Raw Data"""
adata_gene = sc.read_csv('GSE100866_PBMC_vs_flow_10X-RNA_umi.csv').transpose()
adata_protein = sc.read_csv('GSE100866_PBMC_vs_flow_10X-ADT_umi.csv').transpose()

In [ ]:
GSM_Gene_8737 = pd.DataFrame(fmm.mmread('GSM5008737/matrix.mtx.gz').toarray().transpose())
cellNames = pd.DataFrame(pd.read_csv('GSM5008737/barcodes.tsv.gz',index_col=0,sep='\t',header=None))
geneNames = pd.DataFrame(pd.read_csv('GSM5008737/features.tsv.gz',index_col=0,sep='\t',header=None))
GSM_Gene_8737.columns = geneNames.iloc[:,0]
GSM_Gene_8737.index = cellNames.index

In [ ]:
GSM_Protein_8738 = pd.DataFrame(fmm.mmread('GSM5008738/matrix.mtx.gz').toarray().transpose())
cellNames_pro = pd.DataFrame(pd.read_csv('GSM5008738/barcodes.tsv.gz',index_col=0,sep='\t',header=None))
SurfaceProtein = pd.DataFrame(pd.read_csv('GSM5008738/features.tsv.gz',index_col=0,sep='\t',header=None))
GSM_Protein_8738.columns = SurfaceProtein.iloc[:,0]
GSM_Protein_8738.index = cellNames_pro.index

In [ ]:
adata_gene.var["mt"] = adata_gene.var_names.str.startswith("MOUSE_")
adata_gene_filter = adata_gene[:,~adata_gene.var["mt"]]


In [ ]:
np.random.seed(42)
selected_cells = np.random.choice(adata_gene_filter.n_obs, size=6388, replace=False)
remaining_cells = np.setdiff1d(np.arange(adata_gene_filter.n_obs), selected_cells)
adata_gene_train_8737 = adata_gene_filter[selected_cells].copy()
adata_gene_test_8737 = adata_gene_filter[remaining_cells].copy()

In [ ]:
adata_gene_train_8737.X = adata_gene_train_8737.X.astype(int)
adata_gene_train_8737.to_df()

In [ ]:
adata_gene_test_8737.X = adata_gene_test_8737.X.astype(int)
adata_gene_test_8737.to_df()

In [ ]:
np.random.seed(42)
selected_cells = np.random.choice(adata_protein.n_obs, size=6388, replace=False)
remaining_cells_pro = np.setdiff1d(np.arange(adata_protein.n_obs), selected_cells)
adata_protein_train_8783 = adata_protein[selected_cells].copy()
adata_protein_test_8783 = adata_protein[remaining_cells_pro].copy()

In [ ]:
adata_protein_train_8783.X = adata_protein_train_8783.X.astype(int)
adata_protein_train_8783.to_df()

## Scenario 1: Training CITE-seq, Test scRNA-seq

For this scenario, we designate patients P1, P3, P4, P7 as the training data. The other patients are treated as an scRNA-seq test set (proteins are held out).

In [ ]:
"""Create training and test"""

train_bool = [x in ['P1', 'P3', 'P4', 'P7'] for x in adata_gene.obs['donor']]

adata_gene_train = adata_gene[train_bool].copy()
adata_protein_train = adata_protein[train_bool].copy()
adata_gene_test = adata_gene[np.invert(train_bool)].copy()

Create the sciPENN object. Since we only have a single CITE-seq reference, we provide only a single gene training set (represented by the list of length 1 passed to gene_trainsets) and a single protein training set (represented by the list of length 1 passed to protein_trainsets). 

The key used to identify batches is 'donor' for both the CITE-seq reference and the scRNA-seq query, so we pass this key to both the train_batchkeys argument and the test_batchkey argument.

Lastly, we also want to transfer celltype labels from CITE-seq reference to query. The celltypes in the CITE-seq reference are identified by the obs key "celltype.l3", so we supply this string to the type_key argument.

In [ ]:
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_train], protein_trainsets = [adata_protein_train], 
                      gene_test = adata_gene_test, train_batchkeys = ['donor'], test_batchkey = 'donor', 
                      type_key = 'celltype.l3')

In [ ]:


# Filter mitochondrial genes
adata_gene.var["mt"] = adata_gene.var_names.str.startswith("MOUSE_")
adata_gene_filter = adata_gene[:, ~adata_gene.var["mt"]]

# Split data into training and testing sets for gene data
np.random.seed(42)
selected_cells_gene = np.random.choice(adata_gene_filter.n_obs, size=6388, replace=False)
remaining_cells_gene = np.setdiff1d(np.arange(adata_gene_filter.n_obs), selected_cells_gene)
adata_gene_train_8737 = adata_gene_filter[selected_cells_gene].copy()
adata_gene_test_8737 = adata_gene_filter[remaining_cells_gene].copy()

# Ensure data is of integer type
adata_gene_train_8737.X = adata_gene_train_8737.X.astype(int)
adata_gene_test_8737.X = adata_gene_test_8737.X.astype(int)

# Split data into training and testing sets for protein data
np.random.seed(42)
selected_cells_protein = np.random.choice(adata_protein.n_obs, size=6388, replace=False)
remaining_cells_protein = np.setdiff1d(np.arange(adata_protein.n_obs), selected_cells_protein)
adata_protein_train_8783 = adata_protein[selected_cells_protein].copy()
adata_protein_test_8783 = adata_protein[remaining_cells_protein].copy()

# Ensure data is of integer type
adata_protein_train_8783.X = adata_protein_train_8783.X.astype(int)

# Debugging: Print shapes and types to ensure correctness
print(f'Gene train shape: {adata_gene_train_8737.shape}, type: {adata_gene_train_8737.X.dtype}')
print(f'Gene test shape: {adata_gene_test_8737.shape}, type: {adata_gene_test_8737.X.dtype}')
print(f'Protein train shape: {adata_protein_train_8783.shape}, type: {adata_protein_train_8783.X.dtype}')





In [ ]:
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_train_8737], protein_trainsets = [adata_protein_train_8783], 
                      gene_test = adata_gene_test_8737)

Train the sciPENN object. Here, we ask the model to estimate quantiles 0.1, 0.25, 0.75, and 0.9. The weights directory is "pbmc_to_pbmc"

In [ ]:
sciPENN

In [ ]:
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), weights_dir = "pbmc_to_pbmc", load = True)

Use the predict method to predict protein expression and celltype labels in the scRNA-seq test dataset.

In [ ]:
predicted_test = sciPENN.predict()

The predicted protein data is accessible via attribute X.

In [ ]:
predicted_test.X

Use the 'batch' cell metadata field to see which batch a cell is from. E.g. the batch information 'DS-Test P5' means the corresponding cell is from batch P5 of the query dataset.

In [ ]:
predicted_test.obs['batch']

Use the 'transfered cell labels' cell metadata field to see the predicted celltype for each cell. We actually know the true celltype label in this case (found in celltype.l3) so we can view the test accuracy of predictions.

In [ ]:
print(predicted_test.obs['transfered cell labels'])
print(f"\n\n\nTest set accuracy: {(predicted_test.obs['transfered cell labels'] == predicted_test.obs['celltype.l3']).mean():.4f}")

Use the embed command to estimate a lower-dimension latent representation of the data. Both the CITE-seq reference and scRNA-seq query are embedded into a common latent space, which can be further dimension reduced and visualized using UMAP.

In [ ]:
embedding = sciPENN.embed()

The embedding is accessible via attribute X.

In [ ]:
predicted_test.X

Use the 'batch' cell metadata field to see which dataset and batch a cell is from. E.g. the batch information 'DS-Test P5' means the corresponding cell is from batch P5 of the query dataset. 'DS-1 P4' means the corresponding cell is from batch P4 of the first CITE-seq reference.

In [ ]:
embedding.obs['batch']

View estimated quantiles of the data. We estimate a quantile for every protein in every cell, so each of these layers is an array whose rows index cells and columns index proteins.

In [ ]:
q25 = predicted_test.layers['q25']
q75 = predicted_test.layers['q75']
q10 = predicted_test.layers['q10']
q90 = predicted_test.layers['q90']

## Scenario 2: Integrate two CITE-seq datasets

For this scenario, we integrate two separate CITE-seq datasets. The first consists of patients P1, P3, P4, P7, and the other CITE-seq dataset contains other patients. We use sciPENN to handle the case where the protein panels of the two CITE-seq datasets are not identical. Some proteins are availible in only one CITE-seq dataset, other proteins are availible only in the other dataset. The goal here is to recover the missing proteins in each dataset so that the two CITE-seq datasets can be merged

In [ ]:
"""Create training and test"""

train_bool = [x in ['P1', 'P3', 'P4', 'P7'] for x in adata_gene.obs['donor']]

adata_gene_set1 = adata_gene[train_bool].copy()
adata_protein_set1 = adata_protein[train_bool].copy()
adata_gene_set2 = adata_gene[np.invert(train_bool)].copy()
adata_protein_set2 = adata_protein[np.invert(train_bool)].copy()

common_proteins = adata_protein_train.var.index
set1only_proteins = np.random.choice(common_proteins, len(common_proteins)//3, False)
common_proteins = np.setdiff1d(common_proteins, set1only_proteins)
set2only_proteins = np.random.choice(common_proteins, len(common_proteins)//2, False)

set1only_proteins = set(set1only_proteins)
set2only_proteins = set(set2only_proteins)

keep_set1 = [x not in set2only_proteins for x in adata_protein_train.var.index]
keep_set2 = [x not in set1only_proteins for x in adata_protein_train.var.index]

adata_protein_set1 = adata_protein_set1[:, keep_set1].copy()
adata_protein_set2 = adata_protein_set2[:, keep_set2].copy()

Create the sciPENN object. Since we two CITE-seq references and no scRNA-seq query set this time, we provide two gene training sets (one for each CITE-seq reference) and a two protein training sets (the corrresponding protein arrays of the two CITE-seq references). Since we have no scRNA-seq query set, do not provide a gene_test argument.

Since we have two CITE-seq references, we need to provide a list of two batch keys for the train_batchkeys argument, which lists the batchkeys for each of the two references. Since the batchkey is 'donor' for both datasets, the two entries in the list are both "donor". Since we have no query scRNA-seq dataset, do not provide a test_batchkey argument.

For this scenario, we do not want to transfer celltype labels from CITE-seq reference to query. The celltypes is not provided.

In [ ]:
sciPENN = sciPENN_API(gene_trainsets = [adata_gene_set1, adata_gene_set2], 
                      protein_trainsets = [adata_protein_set1, adata_protein_set2], 
                      train_batchkeys = ['donor', 'donor'])

Train the sciPENN object. Here, we ask the model to estimate quantiles 0.1, 0.25, 0.75, and 0.9. The weights directory is "pbmc_to_pbmcINTEGRATE"

In [ ]:
sciPENN.train(quantiles = [0.1, 0.25, 0.75, 0.9], n_epochs = 10000, ES_max = 12, decay_max = 6, 
             decay_step = 0.1, lr = 10**(-3), weights_dir = "pbmc_to_pbmcINTEGRATE", load = True)

Use the impute method to impute missing protein expression in each CITE-seq reference set. Note that this is different from the predict method, which would predict protein expression only for the gene test set and return an array with the same number of rows as the test set. The impute function returns an array of n x p, where n is the sum of the number of cells across all CITE-seq references and p is the union of the sets of proteins across al CITE-seq references. A protein will be imputed for a cell in this array only if it wasn't sequenced for that cell. Otherwise the true sequenced value is provided.

In [ ]:
imputed_test = sciPENN.impute()

The integrated protein reference data is accessible via attribute X.

In [ ]:
imputed_test.X

We can use the 'Dataset' cell metadata field to see which CITE-seq dataset each cell is from. Dataset 1 refers to the first CITE-seq dataset (whose gene data was the first element in the gene_trainsets list and whose protein data was the first element in the protein_trainsets list).

In [ ]:
imputed_test.obs['Dataset']

Each dataset has batches inside it. Use the 'batch' cell metadata field to see which batch a cell is from. E.g. the batch information 'DS-1 P4' means the corresponding cell is from batch P4 of dataset 1.

In [ ]:
imputed_test.obs['batch']

We may want a quick way to identify which proteins were sequenced for a particular dataset. The following commands demontrate how to do this.

In [ ]:
proteins = imputed_test.var.index

proteins1 = proteins[imputed_test.var['Dataset 1']] #get proteins sequenced in Dataset 1
proteins2 = proteins[imputed_test.var['Dataset 2']] #get proteins sequenced in Dataset 1

Recall that the overall array (imputed_test.X) consists of both imputed and true protein values, with imputed values being used when the protein wasn't sequenced. We can separate this array into 4 subarrays:

1. Imputed Protein array in Dataset 1
2. Sequenced Protein array in Dataset 1
3. Imputed Protein array in Dataset 2
4. Sequenced Protein array in Dataset 2

In [ ]:
ds1_cells = imputed_test.obs['Dataset'] == 'Dataset 1'
ds2_cells = imputed_test.obs['Dataset'] == 'Dataset 2'

ds1_pred, ds1_seq = np.invert(imputed_test.var['Dataset 1']), imputed_test.var['Dataset 1']
ds2_pred, ds2_seq = np.invert(imputed_test.var['Dataset 2']), imputed_test.var['Dataset 2']

pred1 = imputed_test[ds1_cells, ds1_pred] #imputed protein array in dataset 1
sequenced1 = imputed_test[ds1_cells, ds1_seq] #sequenced protein array in dataset 1
pred2 = imputed_test[ds2_cells, ds2_pred] #imputed protein array in dataset 2
sequenced2 = imputed_test[ds2_cells, ds2_seq] #sequenced protein array in dataset 2

We can embed the two references into a common latent space. The embedding process is identical to scenario 1.

In [ ]:
embedding = sciPENN.embed()

Use the 'batch' cell metadata field to see which dataset and batch a cell is from. E.g.'DS-1 P4' means the corresponding cell is from batch P4 of the first CITE-seq reference. 'DS-2 P5' means the corresponding cell is from batch P5 of the first CITE-seq reference. 

In [ ]:
embedding.obs['batch']

The 'Dataset' attribute is a quick way to check a cell's source dataset.

In [ ]:
embedding.obs['Dataset']

We can view estimated quantiles just like in Scenario 1. Note that each quantile is estimated for all proteins in. all cells, even when the protein is sequenced in the cell. The quantiles are much more useful for proteins which needed to be imputed for a cell.

In [ ]:
q10_pred = imputed_test[ds1_cells, ds1_pred].layers['q10'] #get q10 for imputed proteins from reference 1
q10_truth = imputed_test[ds1_cells, ds1_seq].layers['q10'] #get q10 for sequenced proteins from reference 1, not useful